# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [2]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)

## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [3]:
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   category           200853 non-null  object        
 1   headline           200853 non-null  object        
 2   authors            200853 non-null  object        
 3   link               200853 non-null  object        
 4   short_description  200853 non-null  object        
 5   date               200853 non-null  datetime64[ns]
 6   combined_text      200853 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 10.7+ MB


In [10]:
df['combined_text'] = df['headline'] + " " + df['authors'] + " " + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [12]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [15]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [36]:
model.train(data, total_examples=model.corpus_count, epochs=30)

(193331519, 232940580)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [37]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [38]:
wv.most_similar('Texas')

[('California', 0.7765558958053589),
 ('Utah', 0.7607255578041077),
 ('Ohio', 0.7349432110786438),
 ('Arkansas', 0.7309521436691284),
 ('Maryland', 0.7298174500465393),
 ('Illinois', 0.7264517545700073),
 ('Oregon', 0.7001676559448242),
 ('Oklahoma', 0.6929706335067749),
 ('Tennessee', 0.6921461820602417),
 ('Pennsylvania', 0.688703179359436)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [39]:
wv.most_similar(negative='Texas')

[('Massacre', 0.41470175981521606),
 ("'Respecting", 0.41424494981765747),
 ('Discards', 0.395779013633728),
 ('Two-Point', 0.3925471603870392),
 ('Guest', 0.39049017429351807),
 ('Dorr', 0.3789527416229248),
 ('Recognize', 0.3750646114349365),
 ('Tragedy', 0.3693958520889282),
 ('Hijacks', 0.36878249049186707),
 ('Tab', 0.36533766984939575)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [40]:
wv['Texas']

array([-0.13885076,  2.145705  ,  1.3831524 , -0.9946145 , -1.386143  ,
        0.44843173, -2.9113214 ,  1.6287093 , -0.604446  , -0.067009  ,
        1.7147717 , -0.8701631 ,  0.7854792 , -1.2608505 , -2.0552397 ,
        0.8649417 , -3.2350888 , -0.5429657 , -1.3350289 ,  0.5477011 ,
       -0.17811534,  4.356794  ,  0.22305156,  0.34879094,  2.4992993 ,
        1.0723827 , -0.58385825, -0.23234771, -0.34722304,  0.298686  ,
       -2.142039  , -0.64689994,  4.321631  ,  0.9945584 , -0.4954077 ,
       -1.3717544 ,  0.34699765, -0.27599758, -1.6219317 ,  2.9946659 ,
       -0.5296245 ,  1.6701224 ,  1.5181658 , -1.5059853 , -1.5367209 ,
       -1.1402996 ,  1.3726232 ,  0.1765614 ,  0.41073683, -0.0811977 ,
        1.0860195 , -3.4195747 , -3.0741954 , -0.12541531, -0.1060296 ,
       -1.6237282 ,  0.00669965,  1.2598609 ,  1.1447197 ,  0.5190377 ,
        1.0005432 ,  1.6672238 , -0.7246242 ,  3.3356228 ,  1.8341869 ,
       -2.4794204 ,  1.883227  , -1.8395661 , -0.6283462 ,  2.99

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [41]:
wv.vectors.shape

(173792, 100)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [44]:
wv.most_similar(positive=['king', 'woman'])

[('man', 0.6954572200775146),
 ('guy', 0.5987337231636047),
 ('girl', 0.5976176857948303),
 ('person', 0.5938334465026855),
 ('boy', 0.5691179633140564),
 ('victim', 0.564918041229248),
 ('queen', 0.5474077463150024),
 ('teenager', 0.5469149351119995),
 ('hero', 0.5366224646568298),
 ('bride', 0.5349336862564087)]

In [43]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])


[('tch', 0.4375379681587219),
 ('Padre', 0.4326673448085785),
 ('Orcia', 0.43202582001686096),
 ('cking', 0.42868518829345703),
 ('ssy', 0.4205703139305115),
 ('tches', 0.41971254348754883),
 ('Beauvoir', 0.4186878502368927),
 ('ker', 0.41341090202331543),
 ('depultures', 0.40976643562316895),
 ('tax-raising', 0.406473845243454)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!